In [176]:
from tqdm import tqdm
import pandas as pd
import pymongo
from pymongo import MongoClient
import seaborn as sns
import requests
import pandas as pd
pd.to_datetime(int('1683105374'), utc=True, unit='s')
# key: addresses, fields can be: total number, max trans/sec per nft, max trans/sec, max buy to list per second (for an nft)

Timestamp('2023-05-03 09:16:14+0000', tz='UTC')

In [267]:
client = MongoClient()
#Pulls all sales converts to pandas
db = client.NFTDB
sales = db.salesCollection
test = db.testdb
record = sales.find({})
df = pd.DataFrame.from_records(record)

In [178]:
db.list_collection_names() #{'_id':{'a':'x','b':'c'},'bo':'co'}

['BotAddress',
 'testdb',
 'salesCollection',
 'completedSlugsTransfers',
 'completedSlugs',
 'NFTStats',
 'completedSlugsListings',
 'addresstoNFT',
 'listingsCollection',
 'transferCollection',
 'slugCollection',
 'addressCollection']

In [268]:
df['timestamp']

0         2023-09-02T22:02:23
1         2023-09-02T10:34:59
2         2023-09-02T03:04:47
3         2023-09-02T02:10:47
4         2023-09-01T21:23:47
                 ...         
591432    2023-06-01T15:42:47
591433    2023-06-01T15:23:59
591434    2023-06-01T14:31:59
591435    2023-06-01T14:23:35
591436    2023-06-01T14:22:35
Name: timestamp, Length: 591437, dtype: object

In [175]:
# test_doc = {'_id':{'a':'x','h':'c'},'bo':'gloo','wee-whiz':'kaka'}
# test.update_one({'_id':test_doc['_id']}, {"$set": test_doc},upsert=True )
# test.find_one({'_id':test_doc['_id']})

{'_id': {'a': 'x', 'h': 'c'}, 'bo': 'gloo', 'wee-whiz': 'kaka'}

In [156]:
listing_collection = db.listingsCollection.create_index([("id", pymongo.ASCENDING), ("token_id", pymongo.ASCENDING), ('timestamp', pymongo.ASCENDING)], unique=True)
# addressCollection = db.addressCollection
# db.addressCollection.create_index([("transaction_id", pymongo.ASCENDING), ("address", pymongo.ASCENDING)], unique=True)
# db.addressCollection.drop()

In [146]:
for doc in db.addressCollection.aggregate([{"$limit": 5}]):
    print(doc)

{'_id': ObjectId('6507c33ef75f3d26de42349f'), 'transaction_id': '0x53194272a87154c17130d2b466b3c7ce432051388d0033ec4ac9c09d4464aa49', 'address': '0xd3f5ee617d302fb17a5cf7fdb749fa638bc9ddcc', 'timestamp': '2023-09-02T22:02:23', 'token_id': '129', 'slug': 'atsnft'}
{'_id': ObjectId('6507c33ef75f3d26de4234a0'), 'transaction_id': '0x53194272a87154c17130d2b466b3c7ce432051388d0033ec4ac9c09d4464aa49', 'address': '0x4fb1136a1122b2312089b0a7ba2111866a6f7b59', 'timestamp': '2023-09-02T22:02:23', 'token_id': '129', 'slug': 'atsnft'}
{'_id': ObjectId('6507c33ef75f3d26de4234a1'), 'transaction_id': '0x4443b2d20a72e6d1cf6a3127f4efbffb2c0d51db35039ac0328bb3c6eff4c912', 'address': '0xcf86b3a45654fd54810ea19ea31b9cf104da349e', 'timestamp': '2023-09-02T10:34:59', 'token_id': '2103', 'slug': 'atsnft'}
{'_id': ObjectId('6507c33ef75f3d26de4234a2'), 'transaction_id': '0x4443b2d20a72e6d1cf6a3127f4efbffb2c0d51db35039ac0328bb3c6eff4c912', 'address': '0xf19869a67f8c741e117b600d57fce2e3e82934a2', 'timestamp': '20

In [275]:
def add_addresses(df):
    #adds timestamp field
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.sort_values(by='timestamp', inplace = True)
    #splits buy and sell into two dbs, then merges on timestamp and finds unique address
    df_address = pd.DataFrame()
    
    df_address['transaction_id'] = list(df['_id']) * 2
    df_address['address'] = list(df['buyer_address']) + list(df['seller_address'])
    df_address['timestamp'] = list(df['timestamp']) * 2
    df_address['timestamp_min'] = list(df['timestamp'].apply(lambda x: x.replace(second=0))) * 2
    df_address['slug'] = list(df['slug']) * 2
    df_address['token_id'] = list(df['token_id']) * 2
    df_address['type'] = ['buy'] * len(list(df['_id'])) + ['sell'] * len(list(df['_id']))
    return df_address
    # address_collection = pd.DataFrame.from_records(db.addressCollection.find({}))
    # pandas.concat([address_collection,df_addresses]).drop_duplicates().reset_index(drop=True)

In [276]:
address_df = add_addresses(pd.DataFrame.from_records(db.salesCollection.find({})))
# pd.set_option('display.max_columns', None)
# print(address_df)
address_df[['address', 'timestamp', 'timestamp_min', 'slug', 'token_id', 'type']]

,address,timestamp,timestamp_min,slug,token_id,type
0,0x7deee0b8aa6d3fc99881e1df461fe51bf2c8108d,2023-01-01 12:05:23,2023-01-01 12:05:00,renga,9802,buy
1,0x669a8cdce1eaddf94456be50348188b29c03b53c,2023-01-01 12:07:35,2023-01-01 12:07:00,thepotatoz,6628,buy
2,0xba26877c2036e1f2baba63bdd5bfe996b050e13c,2023-01-01 12:16:11,2023-01-01 12:16:00,impostors-genesis-aliens,4520,buy
3,0x2d19e2e80db164a08be52e8a0f1c57e3e7425ee8,2023-01-01 12:22:11,2023-01-01 12:22:00,azuki,4055,buy
4,0x22caaeee9f2e34cf68cb1ef54846d518d35c668c,2023-01-01 12:22:59,2023-01-01 12:22:00,azuki,8620,buy
...,...,...,...,...,...,...
1182869,0x622a5b6c4e544a4c085745c4b147d995bb235bbe,2023-09-04 20:52:23,2023-09-04 20:52:00,akidcalledbeast,662,sell
1182870,0xeb745ce75c877ec975d4cad81d54270e46aec33b,2023-09-04 20:53:59,2023-09-04 20:53:00,akidcalledbeast,3060,sell
1182871,0x89177787e8b88b6b95cfcd0501e79bdd05d370ca,2023-09-04 21:05:59,2023-09-04 21:05:00,sproto-gremlins,1859,sell
1182872,0x83e90475dbc318444620cc09491d37ec29746723,2023-09-04 21:07:47,2023-09-04 21:07:00,veefriends-series-2,54329,sell


In [188]:
trans_sec_df = address_df.groupby(['address', 'timestamp'])
for name, group in trans_sec_df:
    if len(group) > 1 and len(group['type'].unique()) > 1:
        print(name, group)
        break

('0x000a3e111b04a7b3ebcd59b8554b224336057a28', Timestamp('2023-02-21 17:29:47'))                                            transaction_id  \
169386  0xfff5585779d0a3981ac3878148a764ed20a08ea5bf28...   
760822  0x1c54978901a0e4b33852752fca0c3cdd7ae273372049...   

                                           address           timestamp  \
169386  0x000a3e111b04a7b3ebcd59b8554b224336057a28 2023-02-21 17:29:47   
760822  0x000a3e111b04a7b3ebcd59b8554b224336057a28 2023-02-21 17:29:47   

                  slug token_id  type  
169386  opepen-edition    13554   buy  
760822  opepen-edition    10878  sell  


In [195]:
trans_sec_df.count()

array([ 1,  2,  4,  3,  8,  6,  5,  9, 10,  7, 20, 14, 13, 17, 22, 21, 19,
       16, 18, 12, 15, 11])

In [237]:
num_transactions_df = address_df.groupby('address')
test_group = num_transactions_df.get_group('0x000a3e111b04a7b3ebcd59b8554b224336057a28').groupby('timestamp')
test_group.apply(lambda x: x[x['type'].isin(['buy'])] and x[x['type'].isin(['sell'])])
# test_group[(test_group['type'].unique() == ['buy', 'sell'])].count()
# test_group[(test_group['type'].isin(['buy', 'sell']))].count()
# test_group[('buy' in test_group['type']) & ('sell' in test_group['type'])]#.count().max()

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [263]:
# with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    # display(num_transactions_df.get_group('0x000a3e111b04a7b3ebcd59b8554b224336057a28'))
# num_transactions_df.get_group('0x000a3e111b04a7b3ebcd59b8554b224336057a28').head()[['address', 'timestamp', 'slug', 'token_id', 'type']]
#.groupby('timestamp')
# for name, group in num_transactions_df.get_group('0x000a3e111b04a7b3ebcd59b8554b224336057a28').groupby('timestamp'):
    # if len(group) > 1:
        # print(group)
# num_transactions_df.get_group('0x000a3e111b04a7b3ebcd59b8554b224336057a28').apply(lambda x: x['timestamp'].replace(second=0), axis=1).groupby('timestamp')
address_df.iloc[0]['timestamp'].month

1

In [240]:
# address_df['timestamp'].apply(lambda x: x.replace(second = 0))
# address_df['timestamp']

0         2023-01-01 12:05:23
1         2023-01-01 12:07:35
2         2023-01-01 12:16:11
3         2023-01-01 12:22:11
4         2023-01-01 12:22:59
                  ...        
1182869   2023-09-04 20:52:23
1182870   2023-09-04 20:53:59
1182871   2023-09-04 21:05:59
1182872   2023-09-04 21:07:47
1182873   2023-09-04 21:08:35
Name: timestamp, Length: 1182874, dtype: datetime64[ns]

In [289]:
type(address_df.groupby('address'))

pandas.core.groupby.generic.DataFrameGroupBy

In [ ]:
def gen_bot_df(df):
    def max_in_group(grouped_df, transaction_types):
        max_elm = 1
        try:
            for name, group in grouped_df:
                # print(group['type'])
                if len(group) > max_elm and transaction_types[0] in group['type'].unique() and transaction_types[1] in group['type'].unique():
                    max_elm = len(group)
        except:
            print(grouped_df)
        return max_elm
        
    destination = db.address_data
    addresses = df.groupby('address')
    for address, address_group in addresses:

        # counts number of transactions for an address
        num_transactions = int(address_group.count()[0])
        
        # for an address groups all buy and sell transactions within the same second: 
        # group is (address, timestamp)
        trans_sec = address_group.groupby('timestamp')
        max_trans_sec = max_in_group(trans_sec, ['buy', 'sell'])
        # max_trans_sec = 1
        # for name, group in trans_sec:
        #     if len(group) > max_trans_sec and 'buy' in group['type'].unique() and 'sell' in group['type'].unique():
        #         max_trans_sec = len(group)

        # for an address groups all buy and sell transactions for the same nft within the same second: 
        # group is (address, slug, token_id, timestamp)
        trans_sec_nft = address_group.groupby(['slug', 'token_id', 'timestamp'])
        max_trans_sec_nft = max_in_group(trans_sec_nft, ['buy', 'sell'])

        # for an address groups all buy and list actions for the same nft within the same second: 
        # group is (address, slug, token_id, timestamp)
        buy_list_sec_nft = address_group.groupby(['slug', 'token_id', 'timestamp'])
        max_buy_list_sec_nft = max_in_group(buy_list_sec_nft, ['buy', 'list'])

        
        trans_min = address_group.groupby('timestamp_min')
        max_trans_min = max_in_group(trans_sec, ['buy', 'sell'])

        trans_min_nft = address_group.groupby(['slug', 'token_id', 'timestamp_min'])
        max_trans_min_nft = max_in_group(trans_min_nft, ['buy', 'sell'])
        
        
        row = {
            '_id': address,
            'num_transactions': num_transactions,
            'max_trans_sec': max_trans_sec,
            'max_trans_sec_nft': max_trans_sec_nft,
            'max_buy_list_sec_nft': max_buy_list_sec_nft,
            'max_trans_min': max_trans_min,
            'max_trans_min_nft': max_trans_min_nft
        }
        print(row)
        destination.update_one({'_id': row['_id']}, {"$set": row}, upsert=True)
    # slug, token_id likely uniquely identifies an NFT but need to make sure that its a buy and a sell also need to max
    # trans_sec_nft = df.groupby(['address', 'slug', 'token_id', 'timestamp']).count()
    # need to make sure that its a buy and a sell also need to max 
    # trans_sec = df.groupby(['address', 'timestamp'])
    # need to do this 
    # buy_list_sec = df.groupby(['address', 'timestamp']).count()
    
gen_bot_df(address_df)

{'_id': '0x00000000000dea4728c92021a0c54ea8c37ac9cc', 'num_transactions': 1, 'max_trans_sec': 1, 'max_trans_sec_nft': 1, 'max_buy_list_sec_nft': 1, 'max_trans_min': 1, 'max_trans_min_nft': 1}
{'_id': '0x00000000005228b791a99a61f36a130d50600106', 'num_transactions': 7, 'max_trans_sec': 1, 'max_trans_sec_nft': 1, 'max_buy_list_sec_nft': 1, 'max_trans_min': 1, 'max_trans_min_nft': 1}
{'_id': '0x0000000000f3e40730ab6aa40e926fcde8a4db53', 'num_transactions': 2, 'max_trans_sec': 1, 'max_trans_sec_nft': 1, 'max_buy_list_sec_nft': 1, 'max_trans_min': 1, 'max_trans_min_nft': 1}
{'_id': '0x0000000001474228ae1223661ef7ab0d89773b0a', 'num_transactions': 4, 'max_trans_sec': 1, 'max_trans_sec_nft': 1, 'max_buy_list_sec_nft': 1, 'max_trans_min': 1, 'max_trans_min_nft': 1}
{'_id': '0x0000000005d8508109bbdc10b508c4291da70102', 'num_transactions': 2, 'max_trans_sec': 1, 'max_trans_sec_nft': 1, 'max_buy_list_sec_nft': 1, 'max_trans_min': 1, 'max_trans_min_nft': 1}
{'_id': '0x0000000009a74af961f452056d5a8

In [23]:
#adds timestamp field
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.sort_values(by='timestamp', inplace = True)
#splits buy and sell into two dbs, then merges on timestamp and finds unique address
df_buy = pd.DataFrame()
df_sell = pd.DataFrame()

# df_buy['id'] = [slug + str(id) for slug, id in zip(df['slug'], df['token_id'])]
df_buy['address'] = df['buyer_address']
df_buy['timestamp'] = df['timestamp']
# df_buy['type'] = ['buy']*len(df['timestamp'])

# df_sell['id'] = [slug + str(id) for slug, id in zip(df['slug'], df['token_id'])]
df_sell['address'] = df['seller_address']
df_sell['timestamp'] = df['timestamp']
# df_sell['type'] = ['sell']*len(df['timestamp'])
df_merged = pd.merge(df_buy,df_sell)
addresses = df_merged['address'].unique()

In [86]:
#Loops through address with buy/sell within 1 second, finds total trans, adds to list if >10 and finds their username
count = []
peeps = 0 # number of addresses with > 10 transactions
user_names = [] 
bot_addy = []
for address in tqdm(addresses):
    # transactions per address
    trans = len(list(sales.find({'seller_address':address}))+list(sales.find({'buyer_address':address})))
    if trans>10:
        record = sales.find_one({'buyer_address':address})
        if record is None:
            record = sales.find_one({'seller_address':address})
            if record is not None:
                user_name = record['seller_username']
        else:
            user_name = record['buyer_username']
        user_names.append(user_name)
        peeps+=1
        count.append(trans)
        bot_addy.append(address)
print(sum(count)) # total number of transactions
print(peeps)
print(sum(i is None for i in user_names)/len(user_names)) #proportion of users with usernames
print(len(bot_addy))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 161/161 [00:00<00:00, 372.90it/s]

50465
149
0.5100671140939598
149


In [45]:
transaction_df = df_buy
grouped_addresses = pd.concat([transaction_df, df_sell], ignore_index=True).groupby('address')

In [51]:
# grouped_addresses.get_group('0xba26877c2036e1f2baba63bdd5bfe996b050e13c').groupby('timestamp').count()
grouped_addresses.get_group('0xba26877c2036e1f2baba63bdd5bfe996b050e13c').groupby(['timestamp','type']).count()

,,transaction_id,address
timestamp,type,,
2023-01-01 12:16:11,buy,1,1
2023-01-01 12:57:35,sell,1,1
2023-01-01 14:45:59,sell,1,1
2023-01-01 14:49:35,sell,1,1
2023-01-01 19:19:11,sell,1,1
...,...,...,...
2023-03-04 20:16:47,sell,1,1
2023-03-05 14:09:23,buy,1,1
2023-03-05 17:12:23,sell,1,1


In [ ]:
#Adds bot address to collection as a unique primary key
pot_bots = db.BotAddresses
for addy in bot_addy:
    pot_bots.insert_one({'_id':addy})